### Ejercicios


#### 1. Agente de aprendizaje por refuerzo profundo con PPO en OpenAI Gym

**Descripción:**  

Implementa un agente que utilice el algoritmo Proximal Policy Optimization (PPO) para resolver un entorno de OpenAI Gym (por ejemplo, "CartPole-v1"). El ejercicio consiste en configurar el entorno, entrenar al agente y evaluar su desempeño.

**Código de referencia (usando `stable-baselines3`):**

```python
import gym
from stable_baselines3 import PPO

# Crear el entorno
env = gym.make("CartPole-v1")

# Inicializar el agente PPO
model = PPO("MlpPolicy", env, verbose=1)

# Entrenar el modelo
model.learn(total_timesteps=100000)

# Guardar el modelo entrenado
model.save("ppo_cartpole")

# Evaluar el agente
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()

env.close()
```


#### 2. Algoritmo basado en DPO (Direct Preference Optimization)

**Descripción:**  
Crea un algoritmo en el que se compare la salida de un modelo para pares de entradas y se optimice directamente la política en función de las preferencias obtenidas (simuladas o reales). Se debe implementar la función de pérdida que minimice la diferencia entre las salidas preferidas y las no preferidas.

**Código de referencia (pseudocódigo con PyTorch):**

```python
import torch
import torch.nn as nn
import torch.optim as optim

# Definir una red simple (por ejemplo, para generar una salida a partir de una entrada)
class PolicyNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PolicyNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
        
    def forward(self, x):
        return self.fc(x)

# Función de pérdida basada en preferencias (simulada)
def dpo_loss(preferred_output, non_preferred_output):
    # Queremos que la salida preferida sea mayor que la no preferida
    # Se puede usar un margen
    margin = 1.0
    loss = torch.mean(torch.clamp(margin - (preferred_output - non_preferred_output), min=0))
    return loss

# Parámetros de ejemplo
input_dim = 10
output_dim = 1
model = PolicyNet(input_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Ejemplo de datos simulados
x_preferred = torch.randn(32, input_dim)
x_non_preferred = torch.randn(32, input_dim)

# Entrenamiento
for epoch in range(100):
    optimizer.zero_grad()
    output_pref = model(x_preferred)
    output_nonpref = model(x_non_preferred)
    loss = dpo_loss(output_pref, output_nonpref)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch} - Loss: {loss.item()}")
```

#### 3. Pipeline de RLHF para afinar un modelo de lenguaje

**Descripción:**  
Crea un pipeline que integre la retroalimentación humana para afinar un modelo de lenguaje. Se deberá recopilar (o simular) feedback para generar datos de preferencia, entrenar un modelo de recompensa y luego ajustar la política del modelo generativo usando un algoritmo de RL (por ejemplo, PPO).

**Código de referencia (fragmento con Hugging Face y pseudocódigo):**

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Cargar un modelo preentrenado (por ejemplo, GPT-2)
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Simular generación de respuestas para una entrada dada
input_text = "El futuro de la inteligencia artificial es"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=50, do_sample=True)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Respuesta generada:", generated_text)

# SUPUESTAMENTE, se recolectan comparaciones de pares de respuestas (esto se simula)
# En la práctica, se usarían datos etiquetados por humanos.
# Luego, se entrena un modelo de recompensa y se utiliza PPO para ajustar el modelo generativo.
# Aquí se muestra un pseudocódigo para la optimización:
"""
for each training step:
    generate multiple responses for an input
    collect human preferences (or simulated preferences)
    compute reward based on preferences using the reward model
    update the generative model parameters using PPO based on the computed rewards
"""
```

#### 4. Modelo de difusión para generación de imágenes

**Descripción:**  
Diseña e implementa un modelo de difusión que transforme gradualmente una muestra de ruido gaussiano en una imagen realista. El ejercicio implica definir el proceso de forward diffusion (añadir ruido progresivamente) y el proceso inverso de denoising.

**Código de referencia (pseudocódigo usando PyTorch):**

```python
import torch
import torch.nn as nn

# Ejemplo de un bloque simple de denoising
class DenoisingBlock(nn.Module):
    def __init__(self, in_channels):
        super(DenoisingBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        )
    
    def forward(self, x):
        return self.conv(x) + x

# Definir un modelo de difusión simple
class DiffusionModel(nn.Module):
    def __init__(self, channels, num_steps):
        super(DiffusionModel, self).__init__()
        self.num_steps = num_steps
        self.denoising_blocks = nn.ModuleList([DenoisingBlock(channels) for _ in range(num_steps)])
    
    def forward(self, noisy_image):
        x = noisy_image
        for block in self.denoising_blocks:
            x = block(x)
        return x

# Proceso de muestreo
def sample_image(model, shape, num_steps):
    # Comenzar con ruido gaussiano
    x = torch.randn(shape)
    for step in range(num_steps):
        # En cada paso, aplicar el bloque de denoising
        x = model(x)
    return x

# Parámetros de ejemplo
channels = 3
num_steps = 5
model = DiffusionModel(channels, num_steps)

# Generar una imagen (por ejemplo, 64x64 RGB)
generated_image = sample_image(model, (1, channels, 64, 64), num_steps)
print("Imagen generada con forma:", generated_image.shape)
```

#### 5. Fine-Tuning de un LLM basado en GPT usando Hugging Face

**Descripción:**  
Realiza el fine-tuning de un modelo de lenguaje basado en GPT (por ejemplo, GPT-2) en un corpus específico. El ejercicio incluye la carga del modelo y tokenizer, la preparación del dataset y la ejecución del entrenamiento.

**Código de referencia (usando `transformers` y `datasets`):**

```python
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Cargar el modelo y tokenizer preentrenados
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Cargar un dataset de ejemplo (puede ser uno personalizado)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Configurar argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
)

# Inicializar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Ejecutar el fine-tuning
trainer.train()
```

#### 6. Análisis comparativo de LLM: GPT, BERT y T5

**Descripción:**  
Implementa un ejercicio que evalúe el desempeño de distintos LLM (por ejemplo, GPT-2, BERT y T5) en tareas como clasificación, generación y comprensión de texto. Se debe comparar la precisión, la coherencia y la eficiencia de cada modelo.

**Código de referencia (usando Hugging Face pipelines):**

```python
from transformers import pipeline

# Inicializar pipelines para cada modelo
gpt_pipeline = pipeline("text-generation", model="gpt2")
bert_pipeline = pipeline("fill-mask", model="bert-base-uncased")
t5_pipeline = pipeline("text2text-generation", model="t5-small")

# Ejemplo de generación con GPT-2
print("GPT-2 Generación:")
print(gpt_pipeline("El futuro de la inteligencia artificial es", max_length=50))

# Ejemplo de enmascaramiento con BERT
print("\nBERT Fill-Mask:")
print(bert_pipeline("La capital de Francia es [MASK]."))

# Ejemplo de transformación con T5
print("\nT5 Text-to-Text:")
print(t5_pipeline("translate English to French: The house is wonderful."))
```


#### 7. Implementación de un sistema RAG (Retrieval Augmented Generation)

**Descripción:**  
Desarrolla un sistema que integre un modelo de recuperación de información con un modelo generativo. El ejercicio debe mostrar cómo, dado un input (por ejemplo, una pregunta), el sistema recupera documentos relevantes y utiliza esa información para generar una respuesta detallada.

**Código de referencia (usando Hugging Face RAG):**

```python
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

# Cargar el tokenizer y el modelo RAG
tokenizer = RagTokenizer.from_pretrained("facebook/rag-sequence-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-sequence-nq", index_name="exact", use_dummy_dataset=True)
model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq", retriever=retriever)

# Definir una pregunta
question = "¿Cuál es la capital de Francia?"
inputs = tokenizer(question, return_tensors="pt")
# Generar una respuesta
outputs = model.generate(**inputs)
print("Respuesta generada:", tokenizer.batch_decode(outputs, skip_special_tokens=True))
```


#### 8. Optimización y destilación de un LLM

**Descripción:**  
Crea un ejercicio avanzado en el que se reduzca el tamaño de un LLM preentrenado (por ejemplo, BERT o GPT-2) mediante técnicas de destilación y cuantización. Evalúa el desempeño del modelo reducido en tareas de NLP.

**Código de referencia (usando Hugging Face Transformers y DistilBERT como ejemplo):**

```python
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments, DistilBertConfig

# Cargar el modelo y tokenizer de DistilBERT (modelo destilado de BERT)
model_name = "distilbert-base-uncased"
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

# Cargar un dataset de ejemplo
from datasets import load_dataset
dataset = load_dataset("glue", "mrpc")

def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Configurar y ejecutar el entrenamiento
training_args = TrainingArguments(
    output_dir="./distilbert-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

trainer.train()

# Se puede aplicar técnicas adicionales de cuantización usando librerías como ONNX Runtime o Hugging Face Optimum
```

#### 9. Uso del modelo LLama y comparación en Hugging Face

**Descripción:**  
Implementa un proyecto en el que utilices el modelo LLama (o una versión similar disponible en Hugging Face) para la generación de texto. El ejercicio debe incluir la carga del modelo, generación de texto a partir de un prompt y la comparación del desempeño frente a otros LLM.

**Código de referencia (usando Hugging Face, asumiendo que existe un modelo LLama):**

```python
from transformers import AutoTokenizer, AutoModelForCausalLM

# Cargar el modelo LLama (reemplazar "model_llama" por el identificador correcto)
model_name = "facebook/llama-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Generar texto a partir de un prompt
prompt = "La tecnología en el siglo XXI ha transformado"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100, do_sample=True)
print("Texto generado:", tokenizer.decode(outputs[0], skip_special_tokens=True))
```


#### 10. Sistema combinado: optimización, destilación y RLHF para afinar un LLM

**Descripción:**  
Construye un sistema que integre optimización de modelos, técnicas de destilación y RLHF para afinar un LLM. El ejercicio consiste en crear un pipeline que recolecte retroalimentación (real o simulada), destile el conocimiento de un modelo grande y optimice la política mediante RL para lograr una mayor alineación con criterios humanos.

**Código de referencia (pseudocódigo que integra componentes):**

```python
# Pseudocódigo integrador

# Paso 1: Cargar el modelo preentrenado grande (por ejemplo, GPT-2) y recopilar retroalimentación
from transformers import AutoModelForCausalLM, AutoTokenizer
model_large = AutoModelForCausalLM.from_pretrained("gpt2-large")
tokenizer = AutoTokenizer.from_pretrained("gpt2-large")

# Simular generación de texto y retroalimentación
prompt = "Explique los beneficios de la inteligencia artificial en la medicina."
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model_large.generate(**inputs, max_length=100)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# Aquí se recopilaría la retroalimentación humana (simulada para este ejercicio)
feedback = "La respuesta es muy completa y precisa."

# Paso 2: Entrenar un modelo de recompensa basado en la retroalimentación (pseudocódigo)
# reward_model = EntrenarModeloDeRecompensa(generated_text, feedback)

# Paso 3: Aplicar destilación para crear un modelo más pequeño (modelo estudiante)
from transformers import DistilGPT2LMHeadModel
model_student = DistilGPT2LMHeadModel.from_pretrained("distilgpt2")

# Paso 4: Optimizar la política del modelo estudiante utilizando RL (por ejemplo, PPO)
# Se integra un ciclo de entrenamiento que utiliza la recompensa estimada para ajustar el modelo
"""
for each training_step:
    generar múltiples salidas con model_student
    evaluar cada salida usando reward_model
    calcular la ventaja y actualizar model_student usando PPO
"""

# Este pseudocódigo ilustra la integración de RLHF, destilación y optimización.
```

In [ ]:
## Tus respuestas